In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import os
import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore") 



# import lightgbm as lgb
# import xgboost as xgb
from scipy.stats import skew
from scipy import stats
from scipy.stats.stats import pearsonr
from scipy.stats import norm
from collections import Counter
from sklearn.linear_model import LinearRegression,LassoCV, Ridge, LassoLarsCV,ElasticNetCV
from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve, train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler, Normalizer

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

sns.set(style='white', context='notebook', palette='deep')


In [5]:

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [6]:
train.shape
test.shape
# Save the 'Id' column
train_ID = train['ID']
test_ID = test['ID']

# Save Output Columns
train_labels = train['target']

# Now drop the 'Id' column since it's unnecessary for the prediction process.
train.drop("ID", axis = 1, inplace = True)
test.drop("ID", axis = 1, inplace = True)

In [8]:
train_features = train.copy()
train_features.drop(labels=['target'], axis=1, inplace=True)
test_features = test.copy()

In [6]:
train_features.shape
test_features.shape

(114393, 131)

In [9]:
train_features=train_features.fillna(-999)
test_features=test_features.fillna(-999)


train_features.drop (labels=['v10', 'v34', 'v50'], axis=1, inplace=True);
test_features.drop (labels=['v10', 'v34', 'v50'], axis=1, inplace=True);

In [10]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [11]:
train_features=train_features.fillna(-999)
test_features=test_features.fillna(-999)

In [12]:
# Feature Processing: Indexing and roud off
refcols=train_features.columns

for elt in refcols:
    if train_features[elt].dtype=='O':
        train_features[elt], temp = pd.factorize(train_features[elt])
        test_features[elt]=temp.get_indexer(test_features[elt])
    else:
        train_features[elt]=train_features[elt].round(5)
        test_features[elt]=test_features[elt].round(5)

In [13]:
train_features.shape

(114321, 128)

In [14]:
test_features.shape


(114393, 128)

In [24]:
selection = SelectKBest(f_classif, k=100).fit(train_features, train_labels)

In [26]:
train_features = selection.transform(train_features)

In [27]:
train_features.shape

(114321, 100)

In [28]:
test_features = selection.transform(test_features)

In [29]:
test_features.shape

(114393, 100)

In [30]:
train_features= pd.DataFrame(train_features);
test_features= pd.DataFrame(test_features);


In [31]:
# Class to predict the probability
class Class_Predict_BNP:

 def Predict_BNP(self,df_train,df_test,target):
    # Prepraing Train Test Set        
    y = target
    X = df_train.values

    # Select Model
    clf = ExtraTreesClassifier(n_estimators=750,max_features=50, 
                               criterion= 'entropy',min_samples_split= 4, 
                               max_depth= 35, min_samples_leaf= 2, 
                               n_jobs = -1, random_state=12)
                               
    clf.fit(X,y)

    # Prediction probability
    value_proba=clf.predict_proba(df_test.values)
    
    # Submission
    submission=pd.read_csv('sample_submission.csv')
    submission.index=submission.ID
    submission.PredictedProb=value_proba[:,1]
    submission.to_csv('BNP_ETC_SUB1.csv', index=False)
    return;

In [33]:
from sklearn.ensemble import ExtraTreesClassifier

In [34]:
a = Class_Predict_BNP()

In [ ]:
a.Predict_BNP(train_features,test_features,train_labels)

We did not get the high accuracy. Hence we will normalize and standardize the data now

In [39]:
#Normalize the data
from sklearn.preprocessing import RobustScaler
transformer = RobustScaler().fit(train_features)
transformer.transform(train_features)
transformer.transform(test_features)



array([[ 8.42335088e-04,  0.00000000e+00,  1.44055820e-03, ...,
         3.12509523e-04,  0.00000000e+00,  1.48447837e-03],
       [-9.98933849e-01,  0.00000000e+00, -9.98420456e-01, ...,
        -9.99276290e-01,  0.00000000e+00, -9.99160198e-01],
       [-5.32310922e-04,  0.00000000e+00,  1.78300436e-03, ...,
         3.02478526e-04,  0.00000000e+00,  4.15044362e-04],
       ...,
       [ 9.90516265e-04,  0.00000000e+00,  1.06811226e-03, ...,
         4.28045829e-04,  0.00000000e+00,  9.68093748e-04],
       [-9.98933849e-01,  0.00000000e+00, -9.98420456e-01, ...,
        -9.99276290e-01,  0.00000000e+00, -9.99160198e-01],
       [ 1.10026048e-03,  0.00000000e+00,  1.50370727e-03, ...,
         2.56409872e-04,  0.00000000e+00,  7.51652705e-04]])

In [40]:
# Standardise the data
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train_features)
scaler.transform(train_features)        
scaler.transform(test_features)        


array([[ 0.87852993, -0.17563716,  0.87859335, ...,  0.85920398,
        -0.44737166,  0.88003699],
       [-1.13759606, -0.17563716, -1.13832461, ..., -1.16237655,
        -0.44737166, -1.13737295],
       [ 0.87575785, -0.17563716,  0.87928413, ...,  0.8591837 ,
        -0.44737166,  0.8778809 ],
       ...,
       [ 0.87882875, -0.17563716,  0.87784205, ...,  0.85943764,
        -0.44737166,  0.8789959 ],
       [-1.13759606, -0.17563716, -1.13832461, ..., -1.16237655,
        -0.44737166, -1.13737295],
       [ 0.87905006, -0.17563716,  0.87872073, ...,  0.85909053,
        -0.44737166,  0.87855954]])

In [41]:
a.Predict_BNP(train_features,test_features,train_labels)